In [1]:
!pip install utils

In [2]:
from scipy.stats import mannwhitneyu
import itertools
import numpy as np
from collections import defaultdict
from typing import Callable, Optional
import csv
import pandas as pd
import os
import utils

import utils


def is_valid_data_file(file_name:str) -> bool:
    return file_name.endswith("json") or file_name.endswith("txt")


def get_mean_for_combinations(df: pd.DataFrame, 
                       independent_variables: list[str], 
                       dependent_variables: list[str]) -> pd.DataFrame:

    # ensure all the columns are present in the df
    for col in independent_variables+dependent_variables:
        if col not in df:
            raise Exception(f"The column {col} is not in the dataframe\n\t(columns are {list(df.columns)})")
    assert(all(col in df for col in independent_variables))
    assert(dependent_variable in df for dependent_variable in dependent_variables)
    
    grouped = df.groupby(independent_variables, dropna=False)[dependent_variables].mean().reset_index()
    
    return grouped


import json
import os


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        problem_name = item["problem_name"]
        pRef_method = item["pRef_method"]
        pRef_size = item["sample_size"]
        #runtime_seconds = item.get("runtime_seconds", None)  # Extract runtime

        # Try to get runtime at either top-level or tree-level
        runtime_seconds_top = item.get("runtime_seconds", None)

        entries = item["results_by_tree"]

        def get_modified_entry(entry):
            entry["problem"] = item["problem_name"]
            entry["pRef_method"] = item["pRef_method"]
            entry["pRef_size"] = item["sample_size"]

            # pick runtime from tree-level if exists; else top-level
            runtime_sec_depth = entry.get("runtime_seconds", runtime_seconds_top)
            entry["runtime_seconds"] = runtime_sec_depth

            errors = entry["results"]
            entry |= errors
            del entry["results"]

            if "order_tree" in entry:
                del entry["order_tree"]

            return entry

        """
        entries = item["results_by_tree"]

        def get_modified_entry(entry):
            entry["problem"] = problem_name
            entry["pRef_method"] = pRef_method
            entry["pRef_size"] = pRef_size
            entry["runtime_seconds"] = runtime_seconds  # Add runtime to each entry

            errors = entry["results"]
            entry = entry | errors
            del entry["results"]

            if "order_tree" in entry:
                del entry["order_tree"]

            return entry
        """
        entries = list(map(get_modified_entry, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]

'''
def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # New logic for configuration data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'ps_budget': tree_setting['ps_budget'],
                        'ps_population': tree_setting['ps_population'],
                        'avoid_ancestors': tree_setting['avoid_ancestors'],
                        'metrics': tree_setting['metrics'],
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # FIXED: Handle different tree types properly
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    # Base entry with common fields
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    
                    # Add tree-type-specific fields
                    if tree_setting['kind'] == 'ps':
                        entry.update({
                            'ps_budget': tree_setting.get('ps_budget', None),
                            'ps_population': tree_setting.get('ps_population', None),
                            'avoid_ancestors': tree_setting.get('avoid_ancestors', None),
                            'metrics': tree_setting.get('metrics', None),
                            'cp': None  # PS doesn't have cp
                        })
                    elif tree_setting['kind'] == 'iai':
                        entry.update({
                            'ps_budget': None,  # IAI doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': tree_setting.get('cp', None)
                        })
                    elif tree_setting['kind'] == 'naive':
                        entry.update({
                            'ps_budget': None,  # Naive doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': None
                        })
                    
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]
'''
def convert_accuracy_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue

        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_entries(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
   
def json_to_tree_data(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        surrounding_information = {
            "problem": item["problem_name"],
            "pRef_method": item["pRef_method"]
        }

        entries = item["results_by_tree"]
        entries = [thing for thing in entries if "order_tree" in thing]

        def convert_order_tree(order_tree, accumulator=None, current_depth=0):
            if accumulator is None:
                accumulator = defaultdict(list)
            accumulator[current_depth].append(order_tree["own"])
            if len(order_tree["matching"]) > 0:
                convert_order_tree(order_tree["matching"], accumulator, current_depth + 1)
            if len(order_tree["unmatching"]) > 0:
                convert_order_tree(order_tree["unmatching"], accumulator, current_depth + 1)
            return accumulator

        def convert_tree_to_averages_by_level(entry):
            ps_search_info = {
                "ps_budget": entry["ps_budget"],
                "ps_population": entry["ps_population"],
                "metrics": entry["metrics"]
            }
            tree_structure = entry["order_tree"]
            just_depths = convert_order_tree(tree_structure)
            core_info_trees = [
                {"depth": depth, "order": order}
                for depth in just_depths
                for order in just_depths[depth]
            ]
            core_info_trees = [surrounding_information | ps_search_info | core_tree for core_tree in core_info_trees]
            return core_info_trees

        entries = list(map(convert_tree_to_averages_by_level, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def convert_tree_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue


        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_tree_data(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    
def filter_dataframe(df, **kwargs):
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original
    for col, value in kwargs.items():
        if col in df.columns:
            df = df[df[col] == value]
        else:
            raise ValueError(f"Column '{col}' not found in dataframe.")
    return df
        

    
    

def prettify_kind_column(df):
    kind_dict = {#"variance":"PS-W",
                 #"variance estimated_atomicity": "PS-WA",
                 "simplicity variance": "PS-SW",
                 "simplicity variance estimated_atomicity" :"PS-SWA"}
    
    df['kind'] = df.apply(
    lambda row: (
        kind_dict[row['metrics']] if row['kind'] == 'ps' else
        'Trad.' if row['kind'] == 'naive' else
        'IAI' if row['kind'] == 'iai' else
        row['kind']
    ),
    axis=1
)
    
    

    
    

In [3]:
import os

In [4]:
#run_location = r"/Users/gian/Desktop/CondorResults/VDT/compareown/run3/"
#run_location = r"C:\Users\gac8\Desktop\CondorResults\VDT\compareown\all_final_runs"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-07-H16'm'47's09"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-10-H01'm'12's39"
#A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H16'm'26's53"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H23'm'35's39"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-20-H01'm'00's52"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-24-H11'm'06's57"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-26-H14'm'06's32"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-27-H12'm'29's47"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\all_final_runs"

#aco and pso
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\all_final_runs"

#bbo aco bro cro
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\ACO+BBO+BRO+CRO"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\all_runs_5"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\all_output_files"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\final_runs_with_time"

run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\all_final_runs_time_again"

results_csv = os.path.join(run_location, "results.csv")
tree_data_csv = os.path.join(run_location, "tree_data.csv")

print("Regenerating CSV files with runtime data...")
convert_accuracy_data_to_df(run_location, results_csv)
convert_tree_data_to_df(run_location, tree_data_csv)

print("CSV files regenerated successfully!")

#convert_accuracy_data_to_df(os.path.join(run_location, "data"), results_csv)
#convert_tree_data_to_df(os.path.join(run_location, "data"), tree_data_csv)



Regenerating CSV files with runtime data...
CSV files regenerated successfully!


In [5]:

accuracy_data = pd.read_csv(results_csv)
prettify_kind_column(accuracy_data)
#tree_data = pd.read_csv(tree_data_csv)

display(accuracy_data)
print(f"\nColumns in accuracy_data: {list(accuracy_data.columns)}")
print(f"\nData types:")
display(accuracy_data.dtypes)

# Check if runtime_seconds column exists and has data
if 'runtime_seconds' in accuracy_data.columns:
    print(f"\nRuntime data available:")
    print(f"Non-null runtime values: {accuracy_data['runtime_seconds'].notna().sum()}")
    print(f"Sample runtime values: {accuracy_data['runtime_seconds'].dropna().head()}")
else:
    print("\nWARNING: runtime_seconds column not found!")

#prettify_kind_column(tree_data)

for kind in accuracy_data["kind"].unique():
    matching_rows = accuracy_data[accuracy_data['kind'] == kind]
    print(f"For the tree kind {kind}, there are {matching_rows.shape[0]} rows")

#headers = "kind,depth,ps_budget,ps_population,avoid_ancestors,metrics,problem,pRef_method,mse,mae,r_sq,evs"

,kind,depth,runtime_seconds,problem,pRef_method,pRef_size,mse,mae,r_sq,evs,ps_budget,ps_population,avoid_ancestors,metrics,cp
0,Trad.,3,1.004,SAT_S,CRO,10000,9.284650,2.400945,0.392605,0.399823,NaN,NaN,NaN,NaN,NaN
1,Trad.,4,1.004,SAT_S,CRO,10000,8.408765,2.304181,0.449905,0.451356,NaN,NaN,NaN,NaN,NaN
2,Trad.,5,1.004,SAT_S,CRO,10000,8.477953,2.199727,0.445378,0.446255,NaN,NaN,NaN,NaN,NaN
3,Trad.,3,2.264,SAT_S,BBO,10000,4.329518,1.745236,0.453288,0.469022,NaN,NaN,NaN,NaN,NaN
4,Trad.,4,2.264,SAT_S,BBO,10000,4.385156,1.679617,0.446262,0.446631,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,PS-SWA,3,62.963,SAT_S,PSO,10000,2.328190,1.168928,0.303515,0.327509,5000.0,100.0,False,simplicity variance estimated_atomicity,NaN
716,PS-SW,4,140.849,SAT_S,PSO,10000,3.149173,1.379821,0.321033,0.325723,5000.0,100.0,False,simplicity variance,NaN
717,PS-SWA,4,114.046,SAT_S,PSO,10000,4.512889,1.542640,0.237185,0.250058,5000.0,100.0,False,simplicity variance estimated_atomicity,NaN
718,PS-SW,5,188.359,SAT_S,PSO,10000,2.639485,1.240152,0.690232,0.706950,5000.0,100.0,False,simplicity variance,NaN



Columns in accuracy_data: ['kind', 'depth', 'runtime_seconds', 'problem', 'pRef_method', 'pRef_size', 'mse', 'mae', 'r_sq', 'evs', 'ps_budget', 'ps_population', 'avoid_ancestors', 'metrics', 'cp']

Data types:


kind                object
depth                int64
runtime_seconds    float64
problem             object
pRef_method         object
pRef_size            int64
mse                float64
mae                float64
r_sq               float64
evs                float64
ps_budget          float64
ps_population      float64
avoid_ancestors     object
metrics             object
cp                 float64
dtype: object


Runtime data available:
Non-null runtime values: 720
Sample runtime values: 0    1.004
1    1.004
2    1.004
3    2.264
4    2.264
Name: runtime_seconds, dtype: float64
For the tree kind Trad., there are 450 rows
For the tree kind PS-SW, there are 90 rows
For the tree kind PS-SWA, there are 90 rows
For the tree kind IAI, there are 90 rows


In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows


In [7]:

def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size=10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    # Check which methods are actually available in your data
    available_methods = usable_data["kind"].unique()
    print(f"Available methods in your data: {available_methods}")
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str):
        # Get data for all available methods
        method_data = {}
        for method in available_methods:
            data = filter_dataframe(usable_data, problem=problem, depth=depth, pRef_method=metaheuristic, kind=method)[result_column]
            if len(data) > 0:  # Only include methods with data
                method_data[method] = data
        
        # If we don't have enough methods to compare, return NaN
        if len(method_data) < 2:
            return {
                "problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "comparison": "insufficient_data",
                "p_value": float('nan'),
                "method_1": None,
                "method_2": None,
                "method_1_mean": float('nan'),
                "method_2_mean": float('nan')
            }
        
        # If we have PS-SW and PS-SWA, compare them
        if "PS-SW" in method_data and "PS-SWA" in method_data:
            p_value = mannwhitneyu(method_data["PS-SW"], method_data["PS-SWA"], alternative="two-sided").pvalue
            comparison = "PS-SW_vs_PS-SWA"
            method_1, method_2 = "PS-SW", "PS-SWA"
            method_1_mean = method_data["PS-SW"].mean()
            method_2_mean = method_data["PS-SWA"].mean()
        else:
            p_value = float('nan')
            comparison = "no_valid_comparison"
            method_1, method_2 = None, None
            method_1_mean, method_2_mean = float('nan'), float('nan')
        
        return {
            "problem": problem,
            "depth": depth,
            "metaheuristic": metaheuristic,
            "comparison": comparison,
            "p_value": p_value,
            "method_1": method_1,
            "method_2": method_2,
            "method_1_mean": method_1_mean,
            "method_2_mean": method_2_mean
        }
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data



In [8]:
    
    
def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size = 10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str) -> (str, np.ndarray):
        for_each_method = {tree_method: filter_dataframe(usable_data, problem = problem, depth = depth, pRef_method = metaheuristic, kind = tree_method)[result_column]
                           for tree_method in {"PS-SW", "PS-SWA", "IAI", "Trad."}}
        
        iai_average = np.average(for_each_method["IAI"])
        naive_average = np.average(for_each_method["Trad."])
        
        
        winning_competitor_method = "IAI" if iai_average > naive_average else "Trad."
        
        p_value_between_w_and_competitor = mannwhitneyu(for_each_method["PS-SW"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        p_value_between_wa_and_competitor = mannwhitneyu(for_each_method["PS-SWA"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        
        return {"problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "p_value_sw":p_value_between_w_and_competitor,
                "p_value_swa": p_value_between_wa_and_competitor,
                "winning_competitor": winning_competitor_method}
    
    
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data["pRef_method"].unique()
    
    dicts = [winning_competitor_for_competition_and_values(problem=problem, depth = depth, metaheuristic=metaheuristic)
             for problem in all_problems
             for depth in depths
             for metaheuristic in all_metaheuristics]
    
    return pd.DataFrame(dicts)
        
        
    

statistical_data = generate_statistical_test_data(accuracy_data, None, None)

pivot_table = statistical_data.pivot_table(index=["problem", "depth", "metaheuristic"], 
                                            values =["p_value_sw", "p_value_swa"])
display(statistical_data)
display(pivot_table)

,problem,depth,metaheuristic,p_value_sw,p_value_swa,winning_competitor
0,SAT_S,3,CRO,0.166667,0.500000,Trad.
1,SAT_S,3,BBO,0.500000,1.000000,IAI
2,SAT_S,3,ACO,0.166667,0.333333,Trad.
3,SAT_S,3,BRO,0.166667,1.000000,Trad.
4,SAT_S,3,PSO,0.500000,0.833333,Trad.
5,SAT_S,4,CRO,0.333333,0.500000,Trad.
6,SAT_S,4,BBO,1.000000,0.500000,IAI
7,SAT_S,4,ACO,0.833333,0.166667,Trad.
8,SAT_S,4,BRO,0.666667,1.000000,Trad.
9,SAT_S,4,PSO,0.833333,1.000000,Trad.


p_value_sw  p_value_swa
problem depth metaheuristic                         
BT      3     ACO              0.166667     0.333333
              BBO              0.166667     0.333333
              BRO              0.500000     0.833333
              CRO              0.666667     0.833333
              PSO              0.500000     1.000000
        4     ACO              0.500000     1.000000
              BBO              0.166667     0.166667
              BRO              0.666667     1.000000
              CRO              0.166667     0.666667
              PSO              1.000000     1.000000
        5     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.166667     0.500000
              PSO              1.000000     0.500000
GC_L    3     ACO              1.000000     0.500000
              BBO              0.500000     0.500000
              BRO              1.000000     0.833333
              CRO              0.166667     0.666667
              PSO              0.500000     0.500000
        4     ACO              0.500000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.500000     0.500000
              PSO              0.500000     0.500000
        5     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.500000     0.500000
              PSO              1.000000     0.500000
GC_S    3     ACO              0.500000     0.500000
              BBO              0.166667     0.166667
              BRO              0.666667     1.000000
              CRO              0.166667     0.333333
              PSO              0.166667     0.166667
        4     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.166667     0.166667
              PSO              0.500000     1.000000
        5     ACO              1.000000     0.500000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              1.000000     0.500000
              PSO              1.000000     0.500000
SAT_L   3     ACO              0.666667     0.166667
              BBO              0.166667     0.166667
              BRO              0.500000     0.500000
              CRO              0.166667     0.166667
              PSO              0.166667     0.166667
        4     ACO              0.166667     0.500000
              BBO              0.500000     0.500000
              BRO              0.500000     0.500000
              CRO              0.166667     0.500000
              PSO              0.500000     0.500000
        5     ACO              0.166667     0.166667
              BBO              0.500000     0.500000
              BRO              1.000000     0.500000
              CRO              0.166667     0.500000
              PSO              0.500000     0.500000
SAT_M   3     ACO              0.166667     0.166667
              BBO              0.166667     0.166667
              BRO              0.166667     1.000000
              CRO              0.500000     0.500000
              PSO              0.166667     0.166667
        4     ACO              0.500000     0.500000
              BBO              0.666667     0.166667
              BRO              0.833333     0.333333
              CRO              0.500000     1.000000
              PSO              0.166667     0.166667
        5     ACO              0.166667     0.166667
              BBO              0.166667     0.666667
              BRO              0.166667     0.666667
              CRO              0.500000     1.000000
 

In [9]:
def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size=10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    time_column = "runtime_seconds"

    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str):
        # Collect R2 and Runtime data for each method
        for_each_method_r2 = {}
        for_each_method_time = {}
        for tree_method in {"PS-SW", "PS-SWA", "IAI", "Trad."}:
            subset = filter_dataframe(usable_data, problem=problem, depth=depth, pRef_method=metaheuristic, kind=tree_method)
            if len(subset) > 0:
                for_each_method_r2[tree_method] = subset[result_column]
                # Check if runtime_seconds column exists and has data
                if time_column in subset.columns and subset[time_column].notna().any():
                    for_each_method_time[tree_method] = subset[time_column]
                else:
                    for_each_method_time[tree_method] = pd.Series([np.nan])  # Use NaN if no runtime data
        
        # If both IAI and Trad. available, decide winner
        iai_average = np.average(for_each_method_r2.get("IAI", [np.nan]))
        trad_average = np.average(for_each_method_r2.get("Trad.", [np.nan]))
        winning_competitor_method = "IAI" if iai_average > trad_average else "Trad."
        
        # Calculate p-values comparing PS-SW and PS-SWA with winning competitor
        if "PS-SW" in for_each_method_r2 and winning_competitor_method in for_each_method_r2:
            p_sw = mannwhitneyu(for_each_method_r2["PS-SW"], for_each_method_r2[winning_competitor_method], alternative="greater").pvalue
        else:
            p_sw = np.nan
        
        if "PS-SWA" in for_each_method_r2 and winning_competitor_method in for_each_method_r2:
            p_swa = mannwhitneyu(for_each_method_r2["PS-SWA"], for_each_method_r2[winning_competitor_method], alternative="greater").pvalue
        else:
            p_swa = np.nan
        
        # ---- Compute runtime averages ----
        mean_times = {m: np.nanmean(v) for m, v in for_each_method_time.items() if len(v) > 0}
        time_sw = mean_times.get("PS-SW", np.nan)
        time_swa = mean_times.get("PS-SWA", np.nan)
        time_iai = mean_times.get("IAI", np.nan)
        time_trad = mean_times.get("Trad.", np.nan)
        
        return {
            "problem": problem,
            "depth": depth,
            "metaheuristic": metaheuristic,
            "winning_competitor": winning_competitor_method,
            "p_value_sw": p_sw,
            "p_value_swa": p_swa,
            "runtime_sw": time_sw,
            "runtime_swa": time_swa,
            "runtime_iai": time_iai,
            "runtime_trad": time_trad
        }
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data["pRef_method"].unique()
    
    dicts = [
        winning_competitor_for_competition_and_values(problem=problem, depth=depth, metaheuristic=metaheuristic)
        for problem in all_problems
        for depth in depths
        for metaheuristic in all_metaheuristics
    ]
    
    return pd.DataFrame(dicts)


In [10]:
# First generate the statistical data
statistical_data = generate_statistical_test_data(accuracy_data, None, None)



# Create the pivot table
pivot_table = statistical_data.pivot_table(
    index=["problem", "depth", "metaheuristic"],
    values=["p_value_sw", "p_value_swa", "runtime_sw", "runtime_swa", "runtime_iai", "runtime_trad"]
)

# Rename the columns for display
pivot_table = pivot_table.rename(columns={
    'runtime_sw': 'execution_time (PS-SW)',
    'runtime_swa': 'execution_time (PS-SWA)'
})

display(pivot_table.round(3))


p_value_sw  p_value_swa  runtime_iai  \
problem depth metaheuristic                                         
BT      3     ACO                 0.167        0.333       49.549   
              BBO                 0.167        0.333       27.232   
              BRO                 0.500        0.833       36.533   
              CRO                 0.667        0.833        7.857   
              PSO                 0.500        1.000       16.348   
        4     ACO                 0.500        1.000       49.549   
              BBO                 0.167        0.167       27.232   
              BRO                 0.667        1.000       36.533   
              CRO                 0.167        0.667        7.857   
              PSO                 1.000        1.000       16.348   
        5     ACO                 1.000        1.000       49.549   
              BBO                 0.500        0.500       27.232   
              BRO                 1.000        1.000       36.533   
              CRO                 0.167        0.500        7.857   
              PSO                 1.000        0.500       16.348   
GC_L    3     ACO                 1.000        0.500       20.481   
              BBO                 0.500        0.500        9.198   
              BRO                 1.000        0.833       24.395   
              CRO                 0.167        0.667        3.469   
              PSO                 0.500        0.500        4.270   
        4     ACO                 0.500        1.000       20.481   
              BBO                 0.500        0.500        9.198   
              BRO                 1.000        1.000       24.395   
              CRO                 0.500        0.500        3.469   
              PSO                 0.500        0.500        4.270   
        5     ACO                 1.000        1.000       20.481   
              BBO                 0.500        0.500        9.198   
              BRO                 1.000        1.000       24.395   
              CRO                 0.500        0.500        3.469   
              PSO                 1.000        0.500        4.270   
GC_S    3     ACO                 0.500        0.500       42.690   
              BBO                 0.167        0.167       19.818   
              BRO                 0.667        1.000       40.331   
              CRO                 0.167        0.333        7.148   
              PSO                 0.167        0.167       10.310   
        4     ACO                 1.000        1.000       42.690   
              BBO                 0.500        0.500       19.818   
              BRO                 1.000        1.000       40.331   
              CRO                 0.167        0.167        7.148   
              PSO                 0.500        1.000       10.310   
        5     ACO                 1.000        0.500       42.690   
              BBO                 0.500        0.500       19.818   
              BRO                 1.000        1.000       40.331   
              CRO                 1.000        0.500        7.148   
              PSO                 1.000        0.500       10.310   
SAT_L   3     ACO                 0.667        0.167       27.645   
              BBO                 0.167        0.167       13.888   
              BRO                 0.500        0.500       15.042   
              CRO                 0.167        0.167        6.062   
              PSO                 0.167        0.167       11.910   
        4     ACO                 0.167        0.500       27.645   
              BBO                 0.500        0.500       13.888   
              BRO                 0.500        0.500       15.042   
              CRO                 0.167        0.500        6.062   
              PSO                 0.500        0.500       11.910   
        5     ACO                 0.167        0.167       27.645   
              BBO          

In [11]:
# Statistical data is now generated in Cell 9
print("Statistical data generated successfully!")
print(f"Shape: {statistical_data.shape}")
print(f"Columns: {list(statistical_data.columns)}")


Statistical data generated successfully!
Shape: (90, 10)
Columns: ['problem', 'depth', 'metaheuristic', 'winning_competitor', 'p_value_sw', 'p_value_swa', 'runtime_sw', 'runtime_swa', 'runtime_iai', 'runtime_trad']


In [12]:
# === New Table with All Runtime Columns ===

pivot_runtime = statistical_data.pivot_table(
    index=["problem", "depth", "metaheuristic"],
    values=["runtime_iai", "runtime_sw", "runtime_swa", "runtime_trad"]
).reset_index()

# Rename columns more descriptively
pivot_runtime.rename(columns={
    "runtime_sw": "Execution Time (PS-SW)",
    "runtime_swa": "Execution Time (PS-SWA)",
    "runtime_iai": "Runtime (IAI)",
    "runtime_trad": "Runtime (Trad.)"
}, inplace=True)

# Display runtime table
display(pivot_runtime)


,problem,depth,metaheuristic,Runtime (IAI),Execution Time (PS-SW),Execution Time (PS-SWA),Runtime (Trad.)
0,BT,3,ACO,49.549,201.414,198.535,44.1910
1,BT,3,BBO,27.232,108.262,128.986,14.3342
2,BT,3,BRO,36.533,92.528,159.507,8.5036
3,BT,3,CRO,7.857,75.611,82.517,4.3024
4,BT,3,PSO,16.348,92.831,93.395,8.2444
5,BT,4,ACO,49.549,252.944,259.284,44.1910
6,BT,4,BBO,27.232,194.563,209.105,14.3342
7,BT,4,BRO,36.533,159.130,254.623,8.5036
8,BT,4,CRO,7.857,138.359,146.684,4.3024
9,BT,4,PSO,16.348,174.929,180.577,8.2444


In [13]:
'''
def bold_max(row):
    row_as_numbers = [float(item[:-1]) for item in row]
    max_number = max(row_as_numbers)
    
    return ['font-weight: bold' if item == max_number else '' for item in row_as_numbers]
'''

'''
def bold_max(row):
    row_as_numbers = []
    for item in row:
        if isinstance(item, str) and item.endswith("%"):
            # Remove % and convert string to float
            row_as_numbers.append(float(item[:-1]))
        else:
            # Already a number (int or float)
            row_as_numbers.append(float(item))
    max_number = max(row_as_numbers)
    return ['font-weight: bold' if num == max_number else '' for num in row_as_numbers]
'''
def bold_max(row):
    numbers = []
    # Only include percentage columns (R²), ignore time columns ending with 's'
    for item in row:
        if isinstance(item, str) and item.endswith("%"):
            try:
                numbers.append(float(item[:-1]))
            except ValueError:
                numbers.append(float("-inf"))
        else:
            numbers.append(float("-inf"))
    if not any([n > float("-inf") for n in numbers]):
        # No percentage columns in this row, don't bold anything
        return ["" for _ in row]
    max_number = max(numbers)
    return ['font-weight: bold' if num == max_number and num > float("-inf") else '' for num in numbers]


def style_pivot_table(pivot_table):
    custom_column_order = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']

    # Check if this is an accuracy table (R² values) or execution time table
    if any('execution_time' in str(col) for col in pivot_table.columns):
        # This is an execution time table - format as seconds
        pivot_table = pivot_table.round(2).astype(str) + "s"
    else:
        # This is an accuracy table - format as percentages
        pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"

    # Reorder columns based on custom order (only for accuracy tables)
    if not any('execution_time' in str(col) for col in pivot_table.columns):
        pivot_table = pivot_table.reindex(columns=custom_column_order)

    styled_df = pivot_table.style.apply(bold_max, axis=1)

    return styled_df

def put_latex_tables_side_by_side(left_latex, right_latex):
    return r"\begin{tabular}{ccccccc}\hline"+left_latex+r"\\ \hline\end{tabular}\quad\begin{tabular}{ccccccc}\hline"+right_latex+r"\\ \hline\end{tabular}"

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "{SA}": r"{\rotcell{SA}}", # note that SA is a subset of SAT\_50 etc.., so it causes some issues
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                    r"\multirow[c]{12}" : r"\hline \multirow[c]{12}",
                    r"& \multirow[c]{3}" : r"\cline{2-7} & \multirow[c]{3}",
                    r"} \cline{2-7}" : "} ",
                   "\\font-weightbold": "",
                    "≪": "\ll "}

    texts_to_rotate = ["problem", "BT", "GC\_anna", "GC\_jean", "SAT\_20",  "SAT\_50",  "SAT\_100", "Met.", "GA", "Tabu", "RS", "PSO", "depth"]  # MOD: added PSO to rotate label in LaTeX output

    for item_to_rotate in texts_to_rotate:
        replacements[item_to_rotate] = r"\rotcell{"+item_to_rotate+"}"

    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string


def pivot_table_as_latex(pivot_table):
    latex_text = pivot_table.to_latex(convert_css=True)
    latex_text = fix_latex(latex_text)
    return latex_text

def create_execution_time_pivot_table(statistical_data):
    """Create a pivot table specifically for execution times"""
    # Create pivot table with execution time columns
    pivot_table = statistical_data.pivot_table(
        index=["problem", "depth", "metaheuristic"],
        values=["runtime_sw", "runtime_swa", "runtime_iai", "runtime_trad"]
    )
    
    # Rename columns for better display
    pivot_table = pivot_table.rename(columns={
        'runtime_sw': 'execution_time (PS-SW)',
        'runtime_swa': 'execution_time (PS-SWA)',
        'runtime_iai': 'execution_time (IAI)',
        'runtime_trad': 'execution_time (Trad.)'
    })
    
    # Apply styling
    styled_df = style_pivot_table(pivot_table)
    return styled_df

In [14]:
def pivot_table_as_latex(pivot_table):
    latex_text = pivot_table.to_latex(convert_css=True)
    latex_text = fix_latex(latex_text)
    return latex_text

In [15]:
# Initialize parameters
pRef_size = 10000
depths = [3, 4, 5]

# Filter data
usable_data = accuracy_data.copy()
usable_data = usable_data[usable_data["pRef_size"] == pRef_size] 
usable_data = usable_data[usable_data["depth"].isin(depths)]

independent_variables = ["problem", "pRef_method", "kind", "depth"]
dependent_variables = ["r_sq"]

problems = ["GC_S", "GC_L", "SAT_S", "SAT_M", "SAT_L", "BT"]
left_problems, right_problems = problems[:3], problems[3:]

def make_combined_table_for_problems(problem_subset):
    methods = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']
    with_right_problems = usable_data[usable_data['problem'].isin(problem_subset)]

    # R2 pivot (flatten columns)
    accuracy_pivot = with_right_problems.pivot_table(
        index=["problem", "pRef_method", "depth"],
        columns="kind",
        values="r_sq"
    )
    # Flatten MultiIndex columns
    if isinstance(accuracy_pivot.columns, pd.MultiIndex):
        accuracy_pivot.columns = [c for c in accuracy_pivot.columns.get_level_values(0)]
    
    # Build runtime table from statistical_data and flatten/index by method
    execution_times = []
    for _, row in statistical_data.iterrows():
        if row['problem'] in problem_subset:
            execution_times.append({
                'problem': row['problem'],
                'pRef_method': row['metaheuristic'],
                'depth': row['depth'],
                'PS-SW': row['runtime_sw'],
                'PS-SWA': row['runtime_swa'],
                'IAI': row['runtime_iai'],
                'Trad.': row['runtime_trad'],
            })
    runtime_df = pd.DataFrame(execution_times)
    runtime_pivot = runtime_df.pivot_table(
        index=["problem", "pRef_method", "depth"],
        values=methods
    )
    runtime_pivot.rename(columns={
        m: f"Execution Time ({m})" for m in methods
    }, inplace=True)

    # Concatenate pivots side by side
    combined = pd.concat([accuracy_pivot, runtime_pivot], axis=1)

    # Interleave columns: [PS-SW, Execution Time (PS-SW), PS-SWA, Execution Time (PS-SWA), ...]
    new_order = []
    for m in methods:
        if m in combined.columns:
            new_order.append(m)
        runtime_col = f"Execution Time ({m})"
        if runtime_col in combined.columns:
            new_order.append(runtime_col)
    combined = combined[new_order]

    # Format columns
    for m in methods:
        if m in combined.columns:
            combined[m] = (combined[m].astype(float) * 100).round(1).astype(str) + '%'
        runtime_col = f"Execution Time ({m})"
        if runtime_col in combined.columns:
            combined[runtime_col] = combined[runtime_col].astype(float).round(2).astype(str) + "s"

    return combined.style.apply(bold_max, axis=1)


# Create tables
left_table = make_combined_table_for_problems(left_problems)
right_table = make_combined_table_for_problems(right_problems)

# Display tables
display(left_table)
display(right_table)

# Add diagnostic information
print("\nDiagnostic Information:")
print("Statistical Data Columns:", statistical_data.columns.tolist())
print("\nSample row from statistical_data:")
print(statistical_data.iloc[0] if len(statistical_data) > 0 else "No data")


left_table_latex = pivot_table_as_latex(left_table)
right_table_latex = pivot_table_as_latex(right_table)

full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

print("left table:")
print(left_table_latex)

print("\n\n\n\n\n\nright table")
print(right_table_latex)






Diagnostic Information:
Statistical Data Columns: ['problem', 'depth', 'metaheuristic', 'winning_competitor', 'p_value_sw', 'p_value_swa', 'runtime_sw', 'runtime_swa', 'runtime_iai', 'runtime_trad']

Sample row from statistical_data:
problem                  SAT_S
depth                        3
metaheuristic              CRO
winning_competitor       Trad.
p_value_sw            0.166667
p_value_swa                0.5
runtime_sw              71.444
runtime_swa             69.895
runtime_iai              4.762
runtime_trad            0.8234
Name: 0, dtype: object
left table:
\begin{tabular}{lllllllllll}
 &  &  & PS-SW & Execution Time (PS-SW) & PS-SWA & Execution Time (PS-SWA) & IAI & Execution Time (IAI) & Trad. & Execution Time (Trad.) \\
\rotcell{problem} & \rotcell{Met.} & \rotcell{depth} &  &  &  &  &  &  &  &  \\
\multirow[c]{15}{*}{\rotcell{GC\_anna}}  & \multirow[c]{3}{*}{ACO} & 3 & 31.5\% & 103.42s & \bfseries 48.6\% & 101.78s & 33.3\% & 20.48s & 31.1\% & 18.04s \\
 &  & 4 & \bf

In [16]:
# Define methods for accuracy and for runtime separately
accuracy_methods = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']
runtime_methods = ['PS-SW', 'PS-SWA']  # exclude IAI and Trad. runtimes

# Create accuracy pivot table that includes all methods
accuracy_pivot = usable_data.pivot_table(
    index=["problem", "pRef_method", "depth"],
    columns="kind",
    values="r_sq"
)

# Flatten MultiIndex columns if present
if isinstance(accuracy_pivot.columns, pd.MultiIndex):
    accuracy_pivot.columns = [c for c in accuracy_pivot.columns.get_level_values(0)]

# Build runtime data only for selected runtime methods
execution_times = []
for _, row in statistical_data.iterrows():
    execution_times.append({
        'problem': row['problem'],
        'pRef_method': row['metaheuristic'],
        'depth': row['depth'],
        'PS-SW': row['runtime_sw'],
        'PS-SWA': row['runtime_swa'],
        # Do not include 'IAI' or 'Trad.' runtime columns here
    })

runtime_df = pd.DataFrame(execution_times)
runtime_pivot = runtime_df.pivot_table(
    index=["problem", "pRef_method", "depth"],
    values=runtime_methods
)
runtime_pivot.rename(columns={
    "PS-SW": "Execution Time (PS-SW)",
    "PS-SWA": "Execution Time (PS-SWA)"
}, inplace=True)

# Concatenate accuracy and runtime pivots horizontally
combined = pd.concat([accuracy_pivot, runtime_pivot], axis=1)

# Interleave columns:
ordered_cols = []
for m in accuracy_methods:
    if m in combined.columns:
        ordered_cols.append(m)
    runtime_col = f"Execution Time ({m})"
    if runtime_col in combined.columns:
        ordered_cols.append(runtime_col)
combined = combined[ordered_cols]

# Format R² as percentages and runtime as seconds
for m in accuracy_methods:
    if m in combined.columns:
        combined[m] = (combined[m].astype(float) * 100).round(1).astype(str) + '%'
    runtime_col = f"Execution Time ({m})"
    if runtime_col in combined.columns:
        combined[runtime_col] = combined[runtime_col].astype(float).round(2).astype(str) + 's'

# Style and display with bold max on R² columns only
styled = combined.style.apply(bold_max, axis=1)
display(styled)


left_table_latex = pivot_table_as_latex(left_table)
right_table_latex = pivot_table_as_latex(right_table)

full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

print("left table:")
print(left_table_latex)

print("\n\n\n\n\n\nright table")
print(right_table_latex)






left table:
\begin{tabular}{lllllllllll}
 &  &  & PS-SW & Execution Time (PS-SW) & PS-SWA & Execution Time (PS-SWA) & IAI & Execution Time (IAI) & Trad. & Execution Time (Trad.) \\
\rotcell{problem} & \rotcell{Met.} & \rotcell{depth} &  &  &  &  &  &  &  &  \\
\multirow[c]{15}{*}{\rotcell{GC\_anna}}  & \multirow[c]{3}{*}{ACO} & 3 & 31.5\% & 103.42s & \bfseries 48.6\% & 101.78s & 33.3\% & 20.48s & 31.1\% & 18.04s \\
 &  & 4 & \bfseries 39.9\% & 168.31s & 30.8\% & 154.84s & 33.3\% & 20.48s & 29.5\% & 18.04s \\
 &  & 5 & 13.6\% & 234.25s & 22.1\% & 215.66s & \bfseries 33.3\% & 20.48s & 21.5\% & 18.04s \\
 \cline{2-7} & \multirow[c]{3}{*}{BBO} & 3 & \bfseries 85.0\% & 68.38s & 80.0\% & 67.53s & 75.9\% & 9.2s & 70.8\% & 4.13s \\
 &  & 4 & \bfseries 90.2\% & 136.68s & 85.7\% & 120.07s & 76.4\% & 9.2s & 75.7\% & 4.13s \\
 &  & 5 & \bfseries 91.8\% & 252.99s & 81.1\% & 224.73s & 81.0\% & 9.2s & 77.9\% & 4.13s \\
 \cline{2-7} & \multirow[c]{3}{*}{BRO} & 3 & -0.5\% & 38.13s & 52.8\% & 91.6s & 45

In [17]:
# Diagnostic checks
print("Statistical Data Info:")
print(statistical_data.info())
print("\nPivot Runtime Info:")
print(pivot_runtime.info())

print("\nSample of Statistical Data:")
print(statistical_data.head())
print("\nSample of Pivot Runtime:")
print(pivot_runtime.head())

Statistical Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   problem             90 non-null     object 
 1   depth               90 non-null     int64  
 2   metaheuristic       90 non-null     object 
 3   winning_competitor  90 non-null     object 
 4   p_value_sw          90 non-null     float64
 5   p_value_swa         90 non-null     float64
 6   runtime_sw          90 non-null     float64
 7   runtime_swa         90 non-null     float64
 8   runtime_iai         90 non-null     float64
 9   runtime_trad        90 non-null     float64
dtypes: float64(6), int64(1), object(3)
memory usage: 7.2+ KB
None

Pivot Runtime Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   ------------

In [18]:
result = []
print(accuracy_data["problem"].unique())
print(accuracy_data["pRef_method"].unique())
for method in accuracy_data["pRef_method"].unique():
    for problem in accuracy_data["problem"].unique():
        for depth in [3, 4, 5]:
            appropriate_data = filter_dataframe(accuracy_data, depth = depth, pRef_method = method, problem = problem, pRef_size=10000)
            trad_data = appropriate_data[appropriate_data["kind"] == "Trad."]["r_sq"][:100]
            own_data = appropriate_data[appropriate_data["kind"] == "PS-SW"]["r_sq"][:100]
            p_value = mannwhitneyu(x = own_data, y = trad_data, alternative="greater").pvalue
            diff = np.average(own_data)-np.average(trad_data)
            
            p_value_string = "0.001" if p_value < 0.001 else round(p_value, 3)
            res = f"{round(diff*100, 2)}%, {p_value_string}"
            if p_value < 0.05:
                res = "\\bfseries "+res
            
            
            result.append({"method":method,
                           "problem":problem,
                           "depth":depth,
                           "res":res})
            
        
improvements = pd.DataFrame(result)
display(improvements)
pivot_table = improvements.pivot_table(index=["problem", "method"],
                                       columns="depth",
                                       values=["res"],
                                       aggfunc=lambda x:x)

#pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

display(pivot_table)

latex_code = pivot_table.to_latex()

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "BT":"Staff R.",
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                   "\\font-weightbold": "",
                    "res": "average R^2 improvemennt between PS-SW and Trad, with p-value",
                    "≪": "\ll "}


    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string
latex_code = fix_latex(latex_code)
print(latex_code)


['SAT_S' 'SAT_M' 'SAT_L' 'GC_S' 'GC_L' 'BT']
['CRO' 'BBO' 'ACO' 'BRO' 'PSO']


,method,problem,depth,res
0,CRO,SAT_S,3,"17.0%, 0.167"
1,CRO,SAT_S,4,"24.79%, 0.333"
2,CRO,SAT_S,5,"29.1%, 0.333"
3,CRO,SAT_M,3,"9.61%, 0.5"
4,CRO,SAT_M,4,"22.68%, 0.333"
5,CRO,SAT_M,5,"23.95%, 0.167"
6,CRO,SAT_L,3,"7.33%, 0.167"
7,CRO,SAT_L,4,"8.73%, 0.167"
8,CRO,SAT_L,5,"7.5%, 0.167"
9,CRO,GC_S,3,"10.85%, 0.167"


res                               
depth                       3               4              5
problem method                                              
BT      ACO      7.99%, 0.167    9.42%, 0.167     1.66%, 0.5
        BBO      8.04%, 0.167    6.41%, 0.167   4.27%, 0.333
        BRO       -0.87%, 0.5   -2.91%, 0.667   -10.65%, 1.0
        CRO     -0.24%, 0.667     4.0%, 0.167   5.53%, 0.167
        PSO     15.69%, 0.167      5.48%, 0.5   6.71%, 0.333
GC_L    ACO      0.39%, 0.667     10.44%, 0.5  -7.94%, 0.667
        BBO     14.24%, 0.167   14.52%, 0.167  13.91%, 0.167
        BRO      -57.65%, 1.0    -15.06%, 1.0   -17.18%, 1.0
        CRO     18.01%, 0.167   16.41%, 0.167  14.67%, 0.333
        PSO     18.27%, 0.167   15.44%, 0.167   -29.74%, 1.0
GC_S    ACO        4.91%, 0.5    -18.09%, 1.0     2.12%, 0.5
        BBO     18.64%, 0.167    9.62%, 0.167   8.74%, 0.167
        BRO     -1.45%, 0.667     -17.5%, 1.0   -17.97%, 1.0
        CRO     10.85%, 0.167    13.3%, 0.167    -0.36%, 0.5
        PSO      12.6%, 0.167    8.85%, 0.167  -1.23%, 0.833
SAT_L   ACO      3.35%, 0.667   10.25%, 0.167   9.16%, 0.167
        BBO     20.12%, 0.167   17.95%, 0.167  17.38%, 0.167
        BRO     17.11%, 0.167   19.98%, 0.167   0.08%, 0.667
        CRO      7.33%, 0.167    8.73%, 0.167    7.5%, 0.167
        PSO     20.78%, 0.167   19.85%, 0.333    10.34%, 0.5
SAT_M   ACO     14.18%, 0.167    9.73%, 0.167   5.45%, 0.167
        BBO      23.3%, 0.167   10.27%, 0.667  19.85%, 0.167
        BRO     15.11%, 0.167   -6.66%, 0.833  12.62%, 0.167
        CRO        9.61%, 0.5   22.68%, 0.333  23.95%, 0.167
        PSO     35.98%, 0.167    23.2%, 0.167  21.88%, 0.167
SAT_S   ACO     14.76%, 0.167  -16.47%, 0.833  -0.22%, 0.667
        BBO     17.45%, 0.333   -1.01%, 0.667  34.72%, 0.167
        BRO     13.32%, 0.167   -0.85%, 0.667  -4.11%, 0.833
        CRO      17.0%, 0.167   24.79%, 0.333   29.1%, 0.333
        PSO        1.53%, 0.5  -15.49%, 0.833  17.67%, 0.167

\begin{tabular}{lllll}
\toprule
 &  & \multicolumn{3}{r}{average R^2 improvemennt between PS-SW and Trad, with p-value} \\
 & depth & 3 & 4 & 5 \\
problem & method &  &  &  \\
\midrule
\multirow[t]{5}{*}{Staff R.} & ACO & 7.99\%, 0.167 & 9.42\%, 0.167 & 1.66\%, 0.5 \\
 & BBO & 8.04\%, 0.167 & 6.41\%, 0.167 & 4.27\%, 0.333 \\
 & BRO & -0.87\%, 0.5 & -2.91\%, 0.667 & -10.65\%, 1.0 \\
 & CRO & -0.24\%, 0.667 & 4.0\%, 0.167 & 5.53\%, 0.167 \\
 & PSO & 15.69\%, 0.167 & 5.48\%, 0.5 & 6.71\%, 0.333 \\
\cline{1-5}
\multirow[t]{5}{*}{GC\_anna} & ACO & 0.39\%, 0.667 & 10.44\%, 0.5 & -7.94\%, 0.667 \\
 & BBO & 14.24\%, 0.167 & 14.52\%, 0.167 & 13.91\%, 0.167 \\
 & BRO & -57.65\%, 1.0 & -15.06\%, 1.0 & -17.18\%, 1.0 \\
 & CRO & 18.01\%, 0.167 & 16.41\%, 0.167 & 14.67\%, 0.333 \\
 & PSO & 18.27\%, 0.167 & 15.44\%, 0.167 & -29.74\%, 1.0 \\
\cline{1-5}
\multirow[t]{5}{*}{GC\_jean} & ACO & 4.91\%, 0.5 & -18.09\%, 1.0 & 2.12\%, 0.5 \\
 & BBO & 18.64\%, 0.167 & 9.62\%, 0.167 & 8.74\%, 0.167 \\
 & BRO & 

In [19]:
# Display runtime data for PS-SW and PS-SWA
print("Runtime Analysis for PS-SW and PS-SWA:")
print("=" * 50)

# Create a focused view on runtime data
runtime_data = statistical_data[['problem', 'depth', 'metaheuristic', 'runtime_sw', 'runtime_swa', 'runtime_iai', 'runtime_trad']].copy()

# Display runtime pivot table
runtime_pivot = runtime_data.pivot_table(
    index=["problem", "depth", "metaheuristic"],
    values=["runtime_sw", "runtime_swa", "runtime_iai", "runtime_trad"]
)

print("\nRuntime Pivot Table (seconds):")
display(runtime_pivot.round(2))

# Create a comparison table showing PS-SW vs PS-SWA runtime
comparison_data = []
for _, row in statistical_data.iterrows():
    if not pd.isna(row['runtime_sw']) and not pd.isna(row['runtime_swa']):
        speedup = row['runtime_swa'] / row['runtime_sw'] if row['runtime_sw'] > 0 else np.nan
        comparison_data.append({
            'problem': row['problem'],
            'depth': row['depth'], 
            'metaheuristic': row['metaheuristic'],
            'PS-SW_runtime': row['runtime_sw'],
            'PS-SWA_runtime': row['runtime_swa'],
            'speedup_ratio': speedup,
            'faster_method': 'PS-SW' if speedup > 1 else 'PS-SWA' if speedup < 1 else 'Equal'
        })

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print("\nPS-SW vs PS-SWA Runtime Comparison:")
    display(comparison_df.round(3))
    
    # Summary statistics
    print(f"\nSummary:")
    print(f"Average speedup (PS-SWA/PS-SW): {comparison_df['speedup_ratio'].mean():.2f}")
    print(f"PS-SW faster in {len(comparison_df[comparison_df['faster_method'] == 'PS-SW'])} cases")
    print(f"PS-SWA faster in {len(comparison_df[comparison_df['faster_method'] == 'PS-SWA'])} cases")
else:
    print("No runtime comparison data available")


Runtime Analysis for PS-SW and PS-SWA:

Runtime Pivot Table (seconds):


runtime_iai  runtime_sw  runtime_swa  \
problem depth metaheuristic                                         
BT      3     ACO                  49.55      201.41       198.54   
              BBO                  27.23      108.26       128.99   
              BRO                  36.53       92.53       159.51   
              CRO                   7.86       75.61        82.52   
              PSO                  16.35       92.83        93.40   
        4     ACO                  49.55      252.94       259.28   
              BBO                  27.23      194.56       209.10   
              BRO                  36.53      159.13       254.62   
              CRO                   7.86      138.36       146.68   
              PSO                  16.35      174.93       180.58   
        5     ACO                  49.55      370.48       355.73   
              BBO                  27.23      329.30       353.42   
              BRO                  36.53      181.66       403.78   
              CRO                   7.86      270.45       300.24   
              PSO                  16.35      298.48       292.07   
GC_L    3     ACO                  20.48      103.42       101.78   
              BBO                   9.20       68.38        67.53   
              BRO                  24.40       38.13        91.60   
              CRO                   3.47       60.51        57.05   
              PSO                   4.27       69.22        62.32   
        4     ACO                  20.48      168.31       154.84   
              BBO                   9.20      136.68       120.07   
              BRO                  24.40      138.10       161.96   
              CRO                   3.47      127.24       142.14   
              PSO                   4.27      133.77       132.86   
        5     ACO                  20.48      234.25       215.66   
              BBO                   9.20      252.99       224.73   
              BRO                  24.40      205.86       290.25   
              CRO                   3.47      224.56       227.81   
              PSO                   4.27      236.00       239.28   
GC_S    3     ACO                  42.69      151.63       154.14   
              BBO                  19.82       81.87        90.42   
              BRO                  40.33       86.41       160.02   
              CRO                   7.15       68.95        73.69   
              PSO                  10.31       70.32        75.55   
        4     ACO                  42.69      236.80       240.06   
              BBO                  19.82      151.45       146.16   
              BRO                  40.33      143.74       260.69   
              CRO                   7.15      142.52       147.96   
              PSO                  10.31      142.20       103.27   
        5     ACO                  42.69      334.56       331.76   
              BBO                  19.82      286.36       301.10   
              BRO                  40.33      293.15       426.44   
              CRO                   7.15      258.12       260.36   
              PSO                  10.31      260.84       272.71   
SAT_L   3     ACO                  27.64      113.05       107.16   
              BBO                  13.89       73.22        80.40   
              BRO                  15.04       81.66        91.33   
              CRO                   6.06       65.64        66.48   
              PSO                  11.91       73.20        74.19   
        4     ACO                  27.64      166.72       164.94   
              BBO                  13.89      136.29       133.81   
              BRO                  15.04      148.01       181.68   
              CRO                   6.06      131.84       138.45   
              PSO                  11.91      133.28       132.75   
        5     ACO                  27.64      288.97       283.92   
              BBO          


PS-SW vs PS-SWA Runtime Comparison:


,problem,depth,metaheuristic,PS-SW_runtime,PS-SWA_runtime,speedup_ratio,faster_method
0,SAT_S,3,CRO,71.444,69.895,0.978,PS-SWA
1,SAT_S,3,BBO,73.874,75.803,1.026,PS-SW
2,SAT_S,3,ACO,66.469,58.189,0.875,PS-SWA
3,SAT_S,3,BRO,65.460,65.282,0.997,PS-SWA
4,SAT_S,3,PSO,52.221,62.963,1.206,PS-SW
5,SAT_S,4,CRO,139.072,115.020,0.827,PS-SWA
6,SAT_S,4,BBO,127.796,112.544,0.881,PS-SWA
7,SAT_S,4,ACO,120.417,104.321,0.866,PS-SWA
8,SAT_S,4,BRO,127.146,111.847,0.880,PS-SWA
9,SAT_S,4,PSO,140.849,114.046,0.810,PS-SWA



Summary:
Average speedup (PS-SWA/PS-SW): 1.07
PS-SW faster in 45 cases
PS-SWA faster in 45 cases


In [20]:
# Diagnostic: Check if runtime data is properly loaded
print("=== RUNTIME DATA DIAGNOSTIC ===")
print(f"Total rows in accuracy_data: {len(accuracy_data)}")
print(f"Columns: {list(accuracy_data.columns)}")

if 'runtime_seconds' in accuracy_data.columns:
    runtime_stats = accuracy_data['runtime_seconds'].describe()
    print(f"\nRuntime statistics:")
    print(runtime_stats)
    
    # Check runtime by method
    print(f"\nRuntime by method:")
    for method in accuracy_data['kind'].unique():
        method_data = accuracy_data[accuracy_data['kind'] == method]['runtime_seconds']
        non_null_count = method_data.notna().sum()
        print(f"  {method}: {non_null_count} non-null values")
        if non_null_count > 0:
            print(f"    Mean: {method_data.mean():.2f}s, Std: {method_data.std():.2f}s")
else:
    print("ERROR: runtime_seconds column not found!")
    print("Available columns:", list(accuracy_data.columns))

# Check a sample of the data
print(f"\nSample data (first 3 rows):")
display(accuracy_data[['problem', 'pRef_method', 'kind', 'depth', 'r_sq', 'runtime_seconds']].head(3))


=== RUNTIME DATA DIAGNOSTIC ===
Total rows in accuracy_data: 720
Columns: ['kind', 'depth', 'runtime_seconds', 'problem', 'pRef_method', 'pRef_size', 'mse', 'mae', 'r_sq', 'evs', 'ps_budget', 'ps_population', 'avoid_ancestors', 'metrics', 'cp']

Runtime statistics:
count    720.000000
mean      47.649557
std       79.833198
min        0.697000
25%        2.549500
50%        7.665000
75%       49.549000
max      426.439000
Name: runtime_seconds, dtype: float64

Runtime by method:
  Trad.: 450 non-null values
    Mean: 7.33s, Std: 9.46s
  PS-SW: 90 non-null values
    Mean: 160.25s, Std: 79.17s
  PS-SWA: 90 non-null values
    Mean: 168.53s, Std: 88.13s
  IAI: 90 non-null values
    Mean: 15.76s, Std: 12.50s

Sample data (first 3 rows):


,problem,pRef_method,kind,depth,r_sq,runtime_seconds
0,SAT_S,CRO,Trad.,3,0.392605,1.004
1,SAT_S,CRO,Trad.,4,0.449905,1.004
2,SAT_S,CRO,Trad.,5,0.445378,1.004


In [21]:
# Create execution time pivot table with styling
print("Execution Time Pivot Table:")
print("=" * 50)

execution_time_table = create_execution_time_pivot_table(statistical_data)
display(execution_time_table)

# Also create LaTeX version
execution_time_latex = pivot_table_as_latex(execution_time_table)
print("\nLaTeX version:")
print(execution_time_latex)


Execution Time Pivot Table:



LaTeX version:
\begin{tabular}{lllllll}
 &  &  & execution_time (IAI) & execution_time (PS-SW) & execution_time (PS-SWA) & execution_time (Trad.) \\
problem & depth & metaheuristic &  &  &  &  \\
\multirow[c]{15}{*}{Staff R.} & \multirow[c]{5}{*}{3} & ACO & 49.55s & 201.41s & 198.54s & 44.19s \\
 &  & BBO & 27.23s & 108.26s & 128.99s & 14.33s \\
 &  & BRO & 36.53s & 92.53s & 159.51s & 8.5s \\
 &  & CRO & 7.86s & 75.61s & 82.52s & 4.3s \\
 &  & PSO & 16.35s & 92.83s & 93.4s & 8.24s \\
 & \multirow[c]{5}{*}{4} & ACO & 49.55s & 252.94s & 259.28s & 44.19s \\
 &  & BBO & 27.23s & 194.56s & 209.1s & 14.33s \\
 &  & BRO & 36.53s & 159.13s & 254.62s & 8.5s \\
 &  & CRO & 7.86s & 138.36s & 146.68s & 4.3s \\
 &  & PSO & 16.35s & 174.93s & 180.58s & 8.24s \\
 & \multirow[c]{5}{*}{5} & ACO & 49.55s & 370.48s & 355.73s & 44.19s \\
 &  & BBO & 27.23s & 329.3s & 353.42s & 14.33s \\
 &  & BRO & 36.53s & 181.66s & 403.78s & 8.5s \\
 &  & CRO & 7.86s & 270.45s & 300.24s & 4.3s \\
 &  & PSO & 16.35s & 2